# Step 16: V13 — Data gaps near transits

Goal: check whether missing data (gaps) within the transit window could be creating or biasing the signal.

Why this matters:
- Gaps can bias depth measurements or create spurious events when folded.
- For long-period candidates with few transits, one missing/partial transit can dominate perception.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V13')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "missing_frac_max": 1.0,
    "missing_frac_max_in_coverage": 0.002,
    "missing_frac_median": 0.001,
    "missing_frac_median_in_coverage": 0.0,
    "n_epochs_evaluated": 10,
    "n_epochs_evaluated_in_coverage": 8,
    "n_epochs_excluded_no_coverage": 2,
    "n_epochs_missing_ge_0p25": 2,
    "n_epochs_missing_ge_0p25_in_coverage": 0,
    "window_mult": 2.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V13: Data gaps
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_data_gaps
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='16_v13_data_gaps')
    run_path = run_out_dir / 'V13_data_gaps.png'
    docs_path = (docs_out_dir / 'V13_data_gaps.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_data_gaps(r, ax=ax)
    ax.set_title('V13: Data gaps near transits')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/16_v13_data_gaps/V13_data_gaps.png`

![V13: Data gaps](../artifacts/tutorial_toi-5807-incremental/16_v13_data_gaps/V13_data_gaps.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/16_v13_data_gaps/V13_data_gaps.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/16_v13_data_gaps/V13_data_gaps.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** 2 epochs have ≥25% missing fraction, but both are outside coverage; in-coverage missing fractions are near zero.
- **Interpretation:** gaps are unlikely to be driving the in-coverage transit signal.
- **Next step:** V15 (transit-window asymmetry).

</details>
